In [178]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calplot
import validators

In [179]:
sns.set_theme()

# Feature Selection for Clustering of Telegram Chats 

# Load the data

We import the data and drop the duplicated index column "Unnamed: 0". 

**Due to the size of the dataset, this might take some time.**

In [180]:
path = os.path.join(os.getcwd(), '../data/csv/freiesth_scrape_2.csv')
df = pd.read_csv(path, low_memory=False)
df.drop(labels="Unnamed: 0", axis=1, inplace=True)

# Initial Exploration

Now, we can start exploring the data by...

1. Inspecting column types.

2. Inspecting column values.

3. Searching the dataframe for missing datapoints.

4. Check for duplicate rows.

5. Verify dates.

6. Verify webpages.

7. Checking the distribution of messages across chats. 

### 1. Inspect column types

First, we inspect the column types to see if they match the data contained in them. To do so, we'll

1. Inspect the datatypes per columns.

2. Check the datatypes in object type columns. 


**Inspect the datatypes per columns**

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1959797 entries, 0 to 1959796
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   chat_handle              object 
 1   chat_name                object 
 2   chat_type                object 
 3   collection_time          object 
 4   fwd_from_chat_handle     object 
 5   fwd_from_chat_id         float64
 6   fwd_from_user_name       object 
 7   is_fwd                   bool   
 8   is_group_elem            bool   
 9   is_reply                 bool   
 10  message_date             object 
 11  message_fwd_count        float64
 12  message_group_id         float64
 13  message_media_type       object 
 14  message_reactions        object 
 15  message_reactions_count  float64
 16  message_text             object 
 17  message_view_count       float64
 18  post_author              float64
 19  reply_to_message_id      float64
 20  reply_to_top_message_id  float64
 21  sender_d

There are some columns that don't match the type of the data contained in them:
- `post_author`: Is `float64`, should be `object`.
- `sender_display_name`: Is `float64`, should be `object`.
- `collection_time`: Is `object`, should be `datetime`.
- `message_date`: Is `object`, should be `datetime`.

**Validate object_type columns**

As the object dtype is a general-purpose dtype for columns with mixed or unknown data types, we need to make sure that they acutally contain the datatype we're looking for.

In our dataset, they should actually contain strings. Let's check, if that's actually the case.



In [182]:
# get the names of all columns of type "object"
object_colums = df.dtypes[df.dtypes == "object"].index.to_list()

# create a dataframe containing the type of each cell
df_object_types =  df[object_colums].map(lambda x: type(x))

# print unique datatype for each column
for column in df_object_types.columns:
    print(f"Types in column \"{column}\"".upper())
    print(df_object_types[column].value_counts())
    print("")

TYPES IN COLUMN "CHAT_HANDLE"
chat_handle
<class 'str'>      1910537
<class 'float'>      49260
Name: count, dtype: int64

TYPES IN COLUMN "CHAT_NAME"
chat_name
<class 'str'>    1959797
Name: count, dtype: int64

TYPES IN COLUMN "CHAT_TYPE"
chat_type
<class 'str'>    1959797
Name: count, dtype: int64

TYPES IN COLUMN "COLLECTION_TIME"
collection_time
<class 'str'>    1959797
Name: count, dtype: int64

TYPES IN COLUMN "FWD_FROM_CHAT_HANDLE"
fwd_from_chat_handle
<class 'float'>    1509382
<class 'str'>       450415
Name: count, dtype: int64

TYPES IN COLUMN "FWD_FROM_USER_NAME"
fwd_from_user_name
<class 'float'>    1953584
<class 'str'>         6213
Name: count, dtype: int64

TYPES IN COLUMN "MESSAGE_DATE"
message_date
<class 'str'>    1959797
Name: count, dtype: int64

TYPES IN COLUMN "MESSAGE_MEDIA_TYPE"
message_media_type
<class 'str'>      1768435
<class 'float'>     191362
Name: count, dtype: int64

TYPES IN COLUMN "MESSAGE_REACTIONS"
message_reactions
<class 'str'>      1182236
<cl

As we can see, some of the columns contain elements of the type `float` -- presumably, those are nan-values, which are saved as floats in pandas.

We will replace them with empty strings down the line. 

### 2. Inspect column values

To get a feeling for the data and to weed out obvious inconsistencies, we can inspect the unqiue values for each column. 

1. First, we'll take a look at the number of unique values per column. This way we can get a feeling for which columns we can reasonably inspect manually.

2. Afterwards, we'll display the  values for each column with less or equal to 20 unique values and inspect them.

In [183]:
unique_values_per_column = df.nunique()
pd.DataFrame(unique_values_per_column, columns=["Unique values"])

,Unique values
chat_handle,4899
chat_name,4969
chat_type,2
collection_time,207393
fwd_from_chat_handle,8168
fwd_from_chat_id,9664
fwd_from_user_name,944
is_fwd,2
is_group_elem,2
is_reply,2


In [184]:
under_20 = unique_values_per_column[unique_values_per_column<=20]

for column in under_20.index:
    print(f"{df[column].value_counts()}\n")

chat_type
broadcast    1893133
megagroup      66664
Name: count, dtype: int64

is_fwd
False    1432670
True      527127
Name: count, dtype: int64

is_group_elem
False    1645573
True      314224
Name: count, dtype: int64

is_reply
False    1886155
True       73642
Name: count, dtype: int64

message_media_type
MessageMediaPhoto               830857
MessageMediaDocumentVideo       569327
MessageMediaWebPage             287285
MessageMediaDocumentAudio        43383
MessageMediaDocumentPhoto        18026
MessageMediaDocumentDocument     16989
MessageMediaPoll                  2315
MessageMediaStory                   86
MessageMediaContact                 44
MessageMediaDocumentOther           40
MessageMediaDocument                26
MessageMediaGeo                     18
MessageMediaGiveaway                18
MessageMediaInvoice                  7
MessageMediaDice                     6
MessageMediaGiveawayResults          4
MessageMediaUnsupported              4
Name: count, dtype: int64


**Findings**:

- As we can see, many columns have a high degree of unique values. This high cardinality is expected due to the inherently variable nature of Telegram messages.


- For some columns with high cardinality, the variability might be influenced by how the data is stored or measured rather than by the content itself. In these cases, we may need to transform or aggregate these columns to make them more suitable for analysis. 
   - For example, datetime columns could be aggregated into broader time periods such as hours, days, or weeks.


- For columns with few unique values, no faulty or obviously inconsistent values have been found.


### 3. Check for missing data

To understand the extent of missing data, we will examine the percentage of missing values for each column in the dataset. This helps us identify which columns have significant amounts of missing data and might require imputation or other handling strategies.


In [185]:
pd.DataFrame(df.isnull().sum().apply(lambda x: x/df.shape[0]).sort_values(ascending=False), columns = ["Missing"])

,Missing
post_author,1.000000
sender_display_name,1.000000
fwd_from_user_name,0.996830
sender_last_name,0.993001
reply_to_top_message_id,0.987378
sender_first_name,0.979443
reply_to_message_id,0.962424
webpage_author,0.949452
webpage_description,0.866623
webpage_title,0.859820


Most of these results are to be expected due to idiosyncrasies of the data collection process, Telegram's API, and the messenger's policy.

For columns with more than 10% values missing, I will provide a brief overview of the reasons why this is the case.

- `sender_display_name`: This value is usually not set because many users do not have a display name.


- `post_author`: This value is only set in certain cases, for example, if an admin of the group sends a message.

- `fwd_from_user_name`: This value is only set if a message was forwarded from a user, which seems to be rare for this dataset.

- `sender_last_name`: This value is only set if a user provided a surname, which seems to be rare for this dataset.

- `reply_to_top_message_id`: This value is only set if a message is a reply in a thread, which seems to be rare for this dataset.

- `sender_first_name`: This value is only set if a user provided a first name, which seems to be rare for this dataset.

- `reply_to_message_id`: This value is only set if a message is a reply to another message, which seems to be rare for this dataset.

- `webpage_author/description/title`: These values are only set for messages that link to a webpage, which provides a preview to Telegram. As both not all messages contain links and not all links provide previews, missing values are to be expected.

- `message_group_id`: This value is only set for messages that are part of a group (for example, photos in an album). As this is not the case for all messages, missing values are to be expected.

- `fwd_from_chat_handle`: This value is only set for messages that were forwarded from another chat. As this is not the case for all messages, missing values are to be expected.

- `fwd_from_chat_id`: Same as above.

- `message_reactions_count`: This value is only set for messages from chats that allow reactions. As this is not the case for all chat types, missing values are to be expected.

- `message_reactions`: This value is only set for messages from chats that allow reactions. As this is not the case for all chat types, missing values are to be expected.

- `message_text`: Some message types, for example, photos in an album or media files, don't contain texts. Missing values are to be expected.

- `message_fwd_count`: This value is only set for messages from chats that provide information on the forwarding counts through the API. As this is not the case for all chat types, missing values are to be expected.

**Conclusion**

- As we can see, most of the missing values actually point towards certain attributes of a message and should be considered in their analysis.

- As they don't contain any relevant information and don't point to relevant information regarding a message, these columns with missing data can be dropped.


### 4. Check for duplicates

Next, we'll check for duplicates.

In [186]:
duplicates  = df[df.duplicated()]
print(f"Duplicates found: {len(duplicates)}")

Duplicates found: 2028


As these duplicates might be referenced in other rows, it might be beneficial to keep them in order to maintain the integrity of these connections.

Let's check if they are referenced in another row. Messages could be both referenced in `reply_to_message_id` or `reply_to_top_message_id`.

In [187]:
duplicate_msg_ids = duplicates["telegram_message_id"]
print(f"Duplicates referenced in `reply_to_messages`: {df["reply_to_message_id"].isin(duplicate_msg_ids).value_counts().iloc[1]}")
print(f"Duplicates referenced in `reply_to_top_message_id`: {df["reply_to_top_message_id"].isin(duplicate_msg_ids).value_counts().iloc[1]}")

Duplicates referenced in `reply_to_messages`: 2888
Duplicates referenced in `reply_to_top_message_id`: 240


As we can see, removing duplicates might lead to the loss of important information regarding reply-chains in our dataset.

We now have two options:

1. If we determine that this information is not needed, we can proceed with dropping the duplicates.

2. Otherwise, we need to be mindful of their potential influence and handle them accordingly.

### 5. Verify dates

Now, let's verify that our dataframe does not contain any major inconsistencies.

While it's impractical to check every single data point, we can make reasonable claims about certain columns, particularly those containing dates and webpage information.

We'll begin by checking if the message_date falls within the expected timeframe. Note that there might be outliers, as messages could have been forwarded to the scraped chat within the timeframe but were originally created outside of it.

To start, we'll visually inspect the times messages were sent using a heatmap.

In [188]:
dates = pd.to_datetime(df["message_date"])

# Create a series with a date index and the message count for each date
messages_dates = dates.dt.date.value_counts().apply(lambda x: x/df.shape[0])
messages_dates.index = pd.to_datetime(messages_dates.index)

# visualize a data
calplot.calplot(messages_dates, cmap='YlGn', colorbar=False) 

**Findings:**
- As we can see, most messages were actually sent during the data-collection timeframe (July 2023 - July 2024).
- As expected, some messages were sent before the data collection timeframe. 
- The messages sent after the data collection period  will be dropped.

### 6. Verify Webpages

The column "urls" contains the urls of webpages referenced in a messages. To prepare them for further analysis down the line, we need to check if they adhere to valid url-formats.

To to so, we'll isolate invalid URLs and evaluate them manually.

In [ ]:
# check, if urls are valid and save results in a list. If a message has no url, we consider it valid.
valid_url = df["webpage_url"].apply(lambda x: validators.url(x) if pd.notnull(x) else True) 

# As validator returns specific error messages, if a message is invalide, we need to replace them with False to use the filter in boolean indexing
invalid_url_filter = [False if elem == True else True for elem in valid_url]

df.loc[invalid_url_filter, ["webpage_url"]]

**Findings**

- Some of the urls contain backslashes, that should be removed.
- Some urls contain double dashes, which causes the url to be falsely flagged as invalid. These cases can be ignored.
- `http://3.US-Militär/	` is actually invalid and should be removed

### 7. Compare the Number of Messages per Chat

Last but not least, let us take a look at some descriptive statistics on how many messages were collected for each chat in our dataset. 

In [ ]:
chats_message_counts = df.groupby("chat_name").size()
chats_message_counts.describe()

**Min & Distribution:** As we can see, the distribution is highly skewed with the majority of chats containing 1 message, while a small number of chats make up a disproportionately large number of messages. This result is to be expected, as a quirk in the data collection process creates a lot of chats with only one message. 

**Max:** The maximum is 100002. This, again, is to be expected, as the data collection process was limited to collect a maximum of 10.000 messages per chat. The two additional messages are presumably messages originating from this chat, that were found as forwarded messages in another chat. In this case, the data-collection software creates an entry for both chats. 

**Conclusion**: As we need a certain amount of content and messages for the vectorisation of a chat, we should drop chats containing only few messages.

### 8. Check how many messages are part of a group

Telegram allows users to send several media files in one message — for example, a photo album. Each file is included as its own message in our dataset, which might interfere with our data analysis.

To decide how to handle them down the line, we'll:
- Check how many messages have a group-ID, which indicates that they are part of an album.

- Confirm that these messages are indeed media files sent as part of an album.

- Examine the average group size to estimate how many messages we'd lose if we choose to drop them.


**Check how many messages are part of the group:**

In [ ]:
print(f"Raw count: {df["message_group_id"].notna().sum()}")
print(f"Percentage of messages: {(df["message_group_id"].notna().sum() / df.shape[0])*100}")

**Confirm that they are messages containing media:**

In [ ]:
# get all grouped messages ant convert their ids to int
grouped_msgs = df[df["message_group_id"].notna()]
grouped_msgs.loc[:, "message_group_id"] = grouped_msgs["message_group_id"].astype(int)

# check their mediatype
print("Media Types of Messages with Group-ID:")
grouped_msgs["message_media_type"].value_counts(dropna=False)

**Calculate the average group size:**

In [ ]:
mean_group_size = np.mean(grouped_msgs.groupby("message_group_id").size())
mean_group_size

**Findings:**

- 16% of all messages are part of a group.

- All of them contain media.

- On average, groups contain between 3 and 4 messages.

**Calculate Estimated Loss if grouped messages are removed:**

As we won't work with images or video (the types of data usually sent in an album) we might only want to keep the message in the album that contains the messages text that was included in the album.

To get an estimate on how many messages we'd lose this way, we'll use the following formula: 

$\text{Estimated Messages Loss} = (\text{Average Group Size} - 1) \times \text{Number of Unique Groups}$


In [ ]:
group_count = grouped_msgs["message_group_id"].nunique()
estimated_message_loss = (mean_group_size-1) * group_count

print(f"Mean group size: {mean_group_size}")
print(f"Estimated messages lost (Raw Count): {int(estimated_message_loss)}")
print(f"Estimated messages lost (Percentage): {estimated_message_loss / df.shape[0] * 100}")

If we drop all grouped messages without a message text, we'll lose about 11% of all messages. 

# Data Cleaning

During the initial exploration we found the following tasks we need to adress before moving on to engineering the features for clustering:

- **Fix the faulty types.**

- **Replace NaN values in object-type columns with empty strings**

- **Drop `sender_display_name` and `post_author` columns**

- **Drop messages sent after the data collection period**

- **Clean urls (remove trailing backslashes and invalid links)**

- **Remove chats containing only few messages**

- **(Optional) Drop grouped messages without text**

Optional tasks include:

- **(Optional) Delete duplicate rows**

- **(Optional) Aggregate datetime columns into broader time periods such as minutes, hours, days, or weeks.**


### 1. Clean Columns

First, we'll drop unnecessary columns and correct any faulty data types.

In [ ]:
# drop columns
df.drop(labels=["sender_display_name","post_author"], axis=1, inplace=True)

# convert columns to datetime
df["collection_time"] = pd.to_datetime(df["collection_time"], errors='coerce')
df["message_date"] = pd.to_datetime(df["message_date"], errors='coerce')

df.info()

### 2. Replace NaN values in object-columns with empty strings. 

Next, we replace NaN values in object columns with empty strings to enable seamless text processing down the line. We'll reuse the object column list created earlier. 

In [ ]:
# get the names of columns of type object
object_column_names = df.dtypes[df.dtypes == "object"].index.to_list()

# fill the NaN-values
df.loc[:, object_column_names] = df.loc[:, object_column_names].fillna('')

Now, we can check, if there are any elements of the type float left.

In [ ]:
# create a dataframe containing the type of each cell
df_object_types =  df[object_column_names].map(lambda x: type(x))

# print unique datatype for each column
for column in df_object_types.columns:
    print(f"Types in column \"{column}\"".upper())
    print(df_object_types[column].value_counts())
    print("")

### 3. Drop Messages sent after the data collection period

Now we drop all messages that were sent after June 2024.

In [ ]:
end_data_collection = pd.Timestamp(year=2024, month=6, day=30)
filter_outliers_date = df["message_date"].dt.date > end_data_collection.date()
df = df[~filter_outliers_date]
print(f"Messages sent after end of data collection left: {(df["message_date"].dt.date > end_data_collection.date()).sum()}")

### 4. Clean URLs

Now we can clean up the urls by removing backslashes and invalid urls.



In [ ]:
# remove trailing backspaces
df["webpage_url"] = df["webpage_url"].str.replace("\\\\$", '', regex=True)

# remove the urls manually evaluated as invalid
invalid_urls = ["http://3.US-Militär/"]
df.loc[df["webpage_url"].isin(invalid_urls) , "webpage_url"] = ''

### 5. Drop duplicates

As we don't plan on using information on connection between messages, we can drop the duplicates. 

In [ ]:
df = df.drop_duplicates()

### 6. Remove Chats containing only few messages

As we need a certain amount of content for the vectorisation and clustering of a chat, we'll drop those with fewer than 1000 available messages. 

Once we're done, we can check the message count statistics for improvements.

In [ ]:
chats_message_counts = df.groupby("chat_name").size()
over_1000 = chats_message_counts[chats_message_counts > 1000]
over_1000_chat_names = list(over_1000.index)
df_over_1000 = df[df["chat_name"].isin(over_1000_chat_names)]

In [ ]:
display(over_1000.describe())
print(f"Messages removed: {abs(df.shape[0] - df_over_1000.shape[0])}")
print(f"Chats removed: {abs(len(chats_message_counts)-len(over_1000))}")
print("")
print(f"Messages remaining: {df_over_1000.shape[0]}")
print(f"Chats remaining: {len(over_1000)}")

After removing 3524 chats containing 147774 messages, we're left with 360 chats with an average message count of 5024 and no fewer than 1013 messages.

# Exploratory Data Analysis

Next, we analyze and visualize the cleaned data to identify features and patterns that might be useful for clustering.



### 1. Compare Chat Types

As the type of chat dictates, shich datapoints are available, we first have to check the distribution of chat-types across our dataset.

Possible chat-types consist of:

- **Broadcast-Channels**: One-to-many communication. There are no replies from Members of this chat, but the API reports view- and forward-counts.

- **Megagroups**: Many-to-many communication. The API does not report detailled interaction-metrics, but there are replies.

To analyse their differences, we'll:

1. Check the distribution of chat-types in our dataset.


2. Look for differences in the available data for each chat type.

In [ ]:
df_over_1000.groupby("telegram_chat_id")["chat_type"].unique().value_counts()

As we can see, most of our channels are Broadcast-Channels.

**Next, let's see if we can find patterns in the available data for each chat.**

We will only keep messages that were forwarded from chats of the same type, as messages forwarded from other chat types might introduce information not natively available in the chat type we are analyzing. We'll use the complete dataset, to maximize the available datapoints. 

In [ ]:
# create a dataframe and dictionary containing all the chat_ids in our initial dataset and the chat id as index
chats_type = df[["telegram_chat_id", "chat_type"]].drop_duplicates()
chats_type = chats_type.set_index("telegram_chat_id")
type_mapping = chats_type["chat_type"].to_dict()

# create a new column containing the type of the source chat
df.loc[:, "source_chat_type"] = df["fwd_from_chat_id"].map(type_mapping)

# replace NaN values in the newly created column to enable comparison with the "chat_type" column
df.loc[:, "source_chat_type"] = df["source_chat_type"].fillna('')

# get all messages that were forwarded from chats
is_fwd = df[(df["fwd_from_user_name"]=='')]
is_fwd = df[df["is_fwd"] == True]

# get all messages not forwarded
is_not_fwd = df[df["is_fwd"] == False]

# keep only messages that are forwarded and have the same source chat and chat type
is_fwd = is_fwd[is_fwd["chat_type"] == is_fwd["source_chat_type"]]

# add the cleaned forwarded messages and the non-forwarded messages back together
cleaned_types = pd.concat([is_fwd, is_not_fwd])

Now, we can compare missing data percentages per column for both types of data.

In [ ]:
# Create seperate DataFrames for each "chat_type".
df_broadcast = cleaned_types[cleaned_types["chat_type"] == "broadcast"]
df_megagroup = cleaned_types[cleaned_types["chat_type"] == "megagroup"]

# Calculate percentage of missing values per column for chats of type "broadcast". Keep only columns with a missing value percentag > 0
missing_values_broadcast = df_broadcast.isna() | (df_broadcast == '')
missing_percentag_broadcast = missing_values_broadcast.sum().apply(lambda x: x/df_broadcast.shape[0])
missing_percentag_broadcast = missing_percentag_broadcast[missing_percentag_broadcast > 0]

# Calculate percentage of missing values per column for chats of type "megagroup". Keep only columns with a missing value percentag > 0
missing_values_megagroup= df_megagroup.isna() | (df_megagroup == '')
missing_percentag_megagroup = missing_values_megagroup.sum().apply(lambda x: x/df_megagroup.shape[0])
missing_percentag_megagroup =missing_percentag_megagroup[missing_percentag_megagroup > 0]

# Combine the two DataFrames into one 
combined_missing_percentages = pd.concat([missing_percentag_broadcast, missing_percentag_megagroup], axis=1)
combined_missing_percentages.columns = ['Missing Values Broadcast', 'Missing Values Megagroup']
combined_missing_percentages

##### Findings:

To check for meaningful differences between chat-types, we'll look at columns with extremly high missing percentages (over 95%) and higher availability in the other:

**message_fwd_count, message_view_count:** 
- Extremly high missing percentage for Broadcasts, as they are not provided by Telegram.

**sender_first_name, sender_last_name**: 
- Only available in Megagroups, as they allow differfent users to send messages. The user recorded for broadcasts has usually only the channels name as a username.

**fwd_from_chat_handle, fwd_from_chat_id, source_chat_type:** 
- Extremly high missing percentages for Megagroups. Indicates that there are few messages forwarded chats in Megagroups.

**reply_to_message_id:**
- Since Broadcasts do not support direct replies from recipients (only the broadcaster sends messages), the reply_to_message_id field is less relevant and often not used. However, it is possible for a broadcaster to forward messages with a reply status from other chats.

**webpage_author, webpage_description, webpage_title, webpage_url:** 
- These fields are more often missing in Megagroups, suggesting a lower number of messages linking to webpages.

**message_group_id:** 
- Low availability for Megagroups suggests, that sending albums of media-files is less common. 

### 2. Inspect Message Type Ratios

Next, we'll analyse the distribution of different message types for each chat. 
We'll focus on the following message-type-ratios across all chats:

- **Forward Ratio**: The proportion of forwarded messages in each chat group.
- **Reply Ratio**: The proportion of replies in each chat group.
- **Webpage Ratio**: The proportion of messages containing a webpage URL in each chat group.
- **Image Ratio**: The proportion of messages containing an image in each chat group.
- **Video Ratio**: The proportion of messages containing a video in each chat group.
- **Document Ratio**: The proportion of messages containing a document in each chat group.

In [ ]:
# create flag-columns 
df_over_1000.loc[:, "is_webpage"] = df_over_1000["webpage_url"].apply(lambda x: False if x == '' else True)
df_over_1000.loc[:, "is_image"] = df_over_1000["message_media_type"].isin(['MessageMediaPhoto','MessageMediaDocumentPhoto'])
df_over_1000.loc[:, "is_video"] = df_over_1000["message_media_type"].isin(['MessageMediaDocumentVideo'])
df_over_1000.loc[:, "is_document"] = df_over_1000["message_media_type"].isin(['MessageMediaDocumentDocument'])

# create a series containing groups sizes
group_sizes = df_over_1000.groupby("telegram_chat_id").size()
# group messages per chat
messages_per_chat = df_over_1000.groupby("telegram_chat_id")

# create a series containing the ratio of forwarded messages per group.
fwd_msg_per_group = messages_per_chat["is_fwd"].sum()
fwd_ratio_per_group = fwd_msg_per_group/group_sizes

# create a series containing the ratio of media-messages per group.
reply_msg_per_group = messages_per_chat["is_reply"].sum()
reply_ratio_per_group = reply_msg_per_group/group_sizes

# create a series containing the ratio of webpage messaages per group.
webpage_msg_per_group = messages_per_chat["is_webpage"].sum()
webpage_ratio_per_group = webpage_msg_per_group/group_sizes

# create a series containing the ratio of image messages per group.
image_msg_per_group = messages_per_chat["is_image"].sum()
image_ratio_per_group = image_msg_per_group/group_sizes

# create a series containing the ratio of video messages per group.
video_msg_per_group = messages_per_chat["is_video"].sum()
video_ratio_per_group = video_msg_per_group/group_sizes

# create a series containing the ratio of video messages per group.
doc_msg_per_group = messages_per_chat["is_document"].sum()
doc_ratio_per_group = doc_msg_per_group/group_sizes

Now, we can compare the distribution of different message types across all chats. 


We'll use a violin plot, as it combines both information on the distribution’s density and spread. 

In [ ]:
data = [fwd_ratio_per_group, reply_ratio_per_group, webpage_ratio_per_group, image_ratio_per_group, video_ratio_per_group, doc_ratio_per_group]
labels = ['Forward Ratio...', 'Reply Ratio...', 'Webpage Ratio...', 'Image Ratio...', 'Video Ratio...', 'Document Ratio...']
dfs = []

# Create a DataFrame for visualisation
for dataset, label in zip(data, labels):
    
    df_ratio = pd.DataFrame({
        'Ratio': dataset,
        'Metric': label
    })
    
    dfs.append(df_ratio)
ratios = pd.concat(dfs, ignore_index=True)

# Create the violin plots
fig, ax = plt.subplots(figsize=(10, 6))
sns.violinplot(x='Metric', y='Ratio', data=ratios, ax=ax, palette='pastel', inner="quart", hue="Metric", legend=False)
ax.grid(True)
fig.subplots_adjust(bottom=0.2)
ax.set_xlabel('...over all chats')
ax.set_ylabel('')
plt.show()

**Forward Ratio**:
- The distribution is broad with a significant number of groups having a forward ratio between 5% and 20%.
- There are a few groups with very high forward ratios (close to 1), indicating that some chats have a high proportion of forwarded messages. These could have the purpose of being "Aggregators".
- The distributions are spread out, which makes forward ration a distinctive feature.

**Reply Ratio:**
- The distribution is very narrow and close to zero for most chat groups.
- This indicates that replies are relatively rare in the chats of our dataset.
- As most of our chats are Broadcast Channels and don't allow replies this is to be expected.

**Webpage Ratio:**
- The distribution shows a moderate density around a ratio of 0 to 0.2.
- This suggests that a small but consistent proportion of messages contain webpage URLs.

**Image Ratio:**
- The distribution is wider and peaks around 0.3 to 0.5, indicating that many chat groups have a substantial proportion of image messages.
- There are also groups with very high image ratios, showing that some chats predominantly share images.

**Video Ratio:**
- The distribution is somewhat similar to the image ratio but slightly lower, peaking around 0.2 to 0.4.
- This indicates that video messages are common but less frequent than image messages.

**Document Ratio:**
- The distribution is very narrow and close to zero for most chat groups.
- This indicates that document messages are quite rare across the groups.
